In [70]:
import numpy as np
import random as rd
import math
import pandas as pd
from numpy import log2 as log
import pprint
eps = np.finfo(float).eps

# class $Dtrees$ is used to Create the dataset for decision tree

In [71]:
class Dtrees:
    def __init__(self,k,m):
        self.k = k    #number of features
        self.m = m    #number of data points
        self.arrofarrX = []
        self.arrY = []
        
        for i in range(self.m):
            new = []
            tmpX = self.generateX()
            tmpW = self.generateW()
            tmpY = self.computeY(tmpX,tmpW)
            self.arrofarrX.append(tmpX)
            self.arrY.append(tmpY)
    
    def ran(self,x,y):
        if rd.uniform(x,y) > 0.5:
            return 1
        else:
            return 0
        
    def ran2(self,x,y):
        if rd.uniform(x,y) > 0.25:
            return 1
        else:
            return 0
        
    def generateX(self):
        new = []
        if self.ran(0,1) == 1:
            new.append(1)
        else:
            new.append(0)
        
        for x in range (1,self.k):
            if self.ran2(0,1) == 1:
                new.append(new[x-1])
            else:
                new.append(1-new[x-1])
        return new
    
    def generateW(self):
        new_w = []
        for x in range (1,self.k):
            de = 0
            for y in range(1,x+1):
                de = de+ math.pow(0.9,y)
            res = math.pow(0.9,x) / de
            new_w.append(res)
        return new_w
    
    def computeY(self,arrX,arrW):
        res = 0
        for x in range (1,self.k):
            res+= arrX[x]*arrW[x-1]
#         print("current result is %d",res)
        if res >= 0.5:
            return arrX[0]
        else:
            return 1 - arrX[0]
        

# Function used to Transfer data into data frame

In [338]:
def insertDS(ds,arr):
    tmp = len(arr)
    for x in range(tmp):
        ds[x] = arr[x]
    return ds

# START BUILDING DECISION TREE

# Compute $H(Y)$

In [599]:
d = Dtrees(20,5000)
df = pd.DataFrame(d.arrofarrX)
L = len(d.arrofarrX[0])
df['Y'] = d.arrY

In [585]:
def find_entropy(df):
    ClFI = df.keys()[-1]   # to get the attribute name of classifcation column
    entropy = 0
    values = df[ClFI].unique()
    for value in values:
        fraction = df[ClFI].value_counts()[value]/len(df[ClFI])
        entropy += -fraction*np.log2(fraction)
    return entropy

# Compute $H(Y|X) = \sum_{x}P(X = x)[-\sum_{y}P(Y=y|X=x)\log P(Y=y|X=x)]$

In [586]:
def find_entropy_attribute(df,attribute):
    ClFI = df.keys()[-1]      
    target_variables = df[ClFI].unique()  
    variables = df[attribute].unique()    
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[ClFI] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

# Find the the attribute that could lead to the biggest entropy

In [587]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]

In [588]:
ans = find_winner(arr)

# get subtable since we find a attribute to cut the table

In [589]:
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)

# (DFS IMPLEMENTATION)FINAL STEP: LETS BUILD DECISION TREE

In [590]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   
    
    node = find_winner(df)   #get the attribute that could lead to he biggest entropy
    
    attValue = np.unique(df[node])   #get the distint attribute in the colomen which lead to the
                                     #biggest entrophy
    
    #init a dictonary structure to store the entire decision tree
    # which is a dictionary of dictionary
    # the innder dictionary used to store the current layer of decision tree
    if tree is None:                    
        tree={}
        tree[node] = {}
    
    #find the index of last column
    
    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Y'],return_counts=True)    #based on the subtable, if counts of unique array
                                                                        #equals one, means only one case exist to decide Y
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #DFS 
                   
    return tree

In [593]:
def predict(pred_arr,tree):     #predict function is just to traverse the TREE using DFS 
    #DFS
    for node in tree.keys():
        value = pred_arr[node]
        tree = tree[node][value]
        ans = 0
        if type(tree) is dict:  # yet to reach the node
            ans = predict(pred_arr,tree)
        else:
            ans = tree
            break;
    return ans

In [594]:
def err_train(df,tree):
    count = 0
    l = len(df)
    for x in range(l):
        inst = df.iloc[x]
        ans = predict(inst,tree)
        if ans == inst[-1]:
            count+=1
        inst = 0
    print((l-count)/l)

In [600]:
err_train(df,tree)

0.0428


In [663]:
def err_train2(df,tree):
    count = 0
    l = len(df)
    for x in range(l):
        inst = df.iloc[x]
        ans = predict(inst,tree)
        if ans == inst[-1]:
            count+=1
        inst = 0
    return (l-count)/l

# Question 1: The class $DTrees$ is used to generate the required dataset based on the description

In [604]:
d = Dtrees(20,10)                #Dtree(20,10) means number of features is 20, and number of data points is 30
df = pd.DataFrame(d.arrofarrX)   #convert the generated dataset of x to dataframe
L = len(d.arrofarrX[0])
df['Y'] = d.arrY                 #concatenate Y to dataframe 

In [605]:
df   #df is the dataframe that contains both $\underline{X}_{m}$ and classification vection $\underline{Y}_m

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,Y
0,1,0,0,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,1
1,1,1,1,0,0,1,1,1,0,1,...,1,0,0,0,1,1,0,0,0,1
2,0,0,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,1,1,1,1,1,...,0,0,0,0,0,1,1,1,0,0
4,0,1,1,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
5,1,1,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
6,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,1
7,0,0,0,1,1,1,0,0,0,1,...,1,1,1,0,0,1,0,0,1,0
8,0,0,0,1,1,1,0,0,0,0,...,1,1,1,1,1,1,0,0,0,0
9,0,1,1,0,0,0,0,0,1,1,...,1,1,1,0,0,0,0,0,1,0


# Question 2: The function to train the Decision Tree is named $buildTree$, it has several helper functions :  {find_{entropy} : used to compute $H(Y)$}, {find_entropy_attribute: used to compute $H(Y|X)$}, {find_winner: used to find the attribute that could lead to the biggest information gain}, and {  get_subtable: used to get subtable based on the attribute input}

In [606]:
tree = buildTree(df)    # call funcion buildTree to train decisiontree

In [607]:
pprint.pprint(tree)    #this is the final decision tree

{0: {0: 0, 1: 1}}


# In question (2) Below I compute the $err_{train}\hat(f)$ which is 0 

In [608]:
err_train(df,tree)    

0.0


# Question(3) First we generate dataset based on number of features is 4, and numer of data points is 30 

In [609]:
d = Dtrees(4,30)                #Dtree(20,10) means number of features is 20, and number of data points is 30
df = pd.DataFrame(d.arrofarrX)   #convert the generated dataset of x to dataframe
L = len(d.arrofarrX[0])
df['Y'] = d.arrY 
df 

,0,1,2,3,Y
0,1,0,0,0,0
1,0,0,0,1,1
2,0,0,0,0,1
3,0,0,0,1,1
4,0,0,0,0,1
5,1,1,1,1,1
6,1,1,1,0,1
7,0,0,1,0,1
8,0,0,0,0,1
9,0,0,0,0,1


# in Question 3, next We train the decision tree based on the input dataset we get right now

In [611]:
tree = buildTree(df)

In [614]:
err_train(df,tree)

0.0


# In question 3, the ordering of variables in decision tree does not make sense, Because, Look at the $ID3$ algorithm, it only select the variable with maximum information gain to split the data, so each time to compute $IG$ our algorithm will traverse all variables.

# Drawing the decision Tree

In [616]:
pprint.pprint(tree) 

{3: {0: {2: {0: {0: {0: {1: {0: 1, 1: 0}}, 1: {1: {0: 0, 1: 1}}}}, 1: 1}},
     1: {0: {0: {1: {0: {2: {0: 1, 1: 0}}, 1: 0}}, 1: {1: {0: 0, 1: 1}}}}}}


In [ ]:
{3: {0: {2: {0: {0: {0: {1: {0: 1, 1: 0}}, 1: {1: {0: 0, 1: 1}}}}, 1: 1}},
     1: {0: {0: {1: {0: {2: {0: 1, 1: 0}}, 1: 0}}, 1: {1: {0: 0, 1: 1}}}}}}

# Below is the result of drawing decision tree based on the tree structure printed above

In [618]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "./decision_tree.png")

# Question 4

# Bascially, I will first generate a dataset to train the decision tree, and generate another dataset to get the training error of decision tree

In [624]:
d = Dtrees(4,30)                #Dtree(4,30) means number of features is 4, and number of data points is 30
df = pd.DataFrame(d.arrofarrX)   #convert the generated dataset of x to dataframe
L = len(d.arrofarrX[0])
df['Y'] = d.arrY 

In [625]:
tree = buildTree(df)
err_train(df,tree)

0.0


# Generate new dataset for testing decision tree:

In [651]:
d = Dtrees(4,50)              
df = pd.DataFrame(d.arrofarrX)   #convert the generated dataset of x to dataframe
L = len(d.arrofarrX[0])
df['Y'] = d.arrY 

# The dataset Below will not been peak by Decision tree before testing and in the process of testing the training error

In [631]:
err_train(df,tree)     

0.03156


# Based on the result, the average error rate of this tree over the newly generated data without any peak from decision tree is $0.03156$

# Question(5)

In [ ]:
err_history = []
count = 0
x_axis = []
for m in range(10,1000):
    d = Dtrees(4,m)              
    df = pd.DataFrame(d.arrofarrX)   #convert the generated dataset of x to dataframe
    L = len(d.arrofarrX[0])
    df['Y'] = d.arrY 
    trainset, predictset = np.split(df, [int(.8*len(df))])   #80% training set, 20% predicting set
    predictset.index = range(len(predictset.index))          #reindex the index of predicting set from zero to len-1
    tree = buildTree(trainset)                               #build decision tree using training set
    errtrain = err_train2(trainset,tree)
    errpred = err_train2(predictset,tree)
    err_history.append(math.fabs(errtrain-errpred))
    x_axis.append(count)
    count = count + 1

# The marginal value in the graph means that in many cases, the error of training set = error of predicting set = 0, in another word, our decision can predict with 100% accurcy in the cases where $|err_{train(\hat{f})-err(\hat{f})| = 0$ , also, we could see that ,this situation may also result from the small value of k which equals 10

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.plot(x_axis, err_history)
